Вам необходимо провести описанные в документе final-statement.html (или final-statement.ipynb) два этапа исследования (для двух подходов к решению задачи), написать по результатам каждого этапа небольшой отчет (ниже указаны вопросы, ответы на которые должны содержаться в отчете), и предоставить для ревью данный отчет и код, с помощью которого вы выполнили задание.

Не забывайте, что в выборке есть признаки, которые "заглядывают в будущее" — они помечены в описании данных как отсутствующие в тестовой выборке. Их прямое использование в модели приведет к переобучению, поэтому не забудьте исключить их из выборки.

In [1]:
import pandas as pd
import numpy as np

## Подход 1: градиентный бустинг "в лоб"

Один из самых универсальных алгоритмов, изученных в нашем курсе, является градиентный бустинг. Он не очень требователен к данным, восстанавливает нелинейные зависимости, и хорошо работает на многих наборах данных, что и обуславливает его популярность. В данном разделе предлагается попробовать градиентный бустинг для решения нашей задачи.

В отчете по данному этапу должны содержаться ответы на следующие вопросы:


1. Какие признаки имеют пропуски среди своих значений? Что могут означать пропуски в этих признаках (ответьте на этот вопрос для двух любых признаков)?
2. Как называется столбец, содержащий целевую переменную?
3. Как долго проводилась кросс-валидация для градиентного бустинга с 30 деревьями? Инструкцию по измерению времени можно найти ниже по тексту. Какое качество при этом получилось? Напомним, что в данном задании мы используем метрику качества AUC-ROC.
4. Имеет ли смысл использовать больше 30 деревьев в градиентном бустинге? Что бы вы предложили делать, чтобы ускорить его обучение при увеличении количества деревьев?


1. Считайте таблицу с признаками из файла features.csv с помощью кода, приведенного выше. Удалите признаки, связанные с итогами матча (они помечены в описании данных как отсутствующие в тестовой выборке).

In [2]:
train_X = pd.read_csv('features.csv')
features_test = pd.read_csv('features_test.csv')

# train_y = train_X[['duration', 'radiant_win', 'tower_status_radiant', 
#                       'tower_status_dire', 'barracks_status_radiant', 'barracks_status_dire']]
train_y = train_X[['radiant_win']]
train_X = train_X.drop(['duration', 'radiant_win', 'tower_status_radiant', 
                      'tower_status_dire', 'barracks_status_radiant', 'barracks_status_dire'], axis=1)

In [3]:
train_X.head()

,match_id,start_time,lobby_type,r1_hero,r1_level,r1_xp,r1_gold,r1_lh,r1_kills,r1_deaths,...,radiant_ward_sentry_count,radiant_first_ward_time,dire_bottle_time,dire_courier_time,dire_flying_courier_time,dire_tpscroll_count,dire_boots_count,dire_ward_observer_count,dire_ward_sentry_count,dire_first_ward_time
0,0,1430198770,7,11,5,2098,1489,20,0,0,...,0,35.0,103.0,-84.0,221.0,3,4,2,2,-52.0
1,1,1430220345,0,42,4,1188,1033,9,0,1,...,0,-20.0,149.0,-84.0,195.0,5,4,3,1,-5.0
2,2,1430227081,7,33,4,1319,1270,22,0,0,...,1,-39.0,45.0,-77.0,221.0,3,4,3,1,13.0
3,3,1430263531,1,29,4,1779,1056,14,0,0,...,0,-30.0,124.0,-80.0,184.0,0,4,2,0,27.0
4,4,1430282290,7,13,4,1431,1090,8,1,0,...,0,46.0,182.0,-80.0,225.0,6,3,3,0,-16.0


In [4]:
train_X.shape

(97230, 103)

In [5]:
train_y.head()

,radiant_win
0,1
1,1
2,0
3,0
4,0


In [6]:
train_y.shape

(97230, 1)

In [7]:
features_test = pd.read_csv('features_test.csv')

2. Проверьте выборку на наличие пропусков с помощью функции count(), которая для каждого столбца показывает число заполненных значений. Много ли пропусков в данных? Запишите названия признаков, имеющих пропуски, и попробуйте для любых двух из них дать обоснование, почему их значения могут быть пропущены.

In [8]:
train_X.count().max()

97230

In [9]:
train_X.count()

match_id                    97230
start_time                  97230
lobby_type                  97230
r1_hero                     97230
r1_level                    97230
                            ...  
dire_tpscroll_count         97230
dire_boots_count            97230
dire_ward_observer_count    97230
dire_ward_sentry_count      97230
dire_first_ward_time        95404
Length: 103, dtype: int64

In [10]:
train_X.isnull().sum().to_frame().rename(columns={0: 'empty'}).query('empty > 0')  # количество пропущенных значений

,empty
first_blood_time,19553
first_blood_team,19553
first_blood_player1,19553
first_blood_player2,43987
radiant_bottle_time,15691
radiant_courier_time,692
radiant_flying_courier_time,27479
radiant_first_ward_time,1836
dire_bottle_time,16143
dire_courier_time,676


- Признак first_blood_time содержит пустые значения, т.к событие "первая кровь" не произошло за первые 5 минут.
- Признак first_blood_team также содержит пустые значения, т.к событие "первая кровь" не произошло за первые 5 минут.

3. Замените пропуски на нули с помощью функции fillna(). На самом деле этот способ является предпочтительным для логистической регрессии, поскольку он позволит пропущенному значению не вносить никакого вклада в предсказание. Для деревьев часто лучшим вариантом оказывается замена пропуска на очень большое или очень маленькое значение — в этом случае при построении разбиения вершины можно будет отправить объекты с пропусками в отдельную ветвь дерева. Также есть и другие подходы — например, замена пропуска на среднее значение признака. Мы не требуем этого в задании, но при желании попробуйте разные подходы к обработке пропусков и сравните их между собой.

In [11]:
train_X = train_X.fillna(0)

In [12]:
train_X.isnull().sum().to_frame().rename(columns={0: 'empty'}).query('empty > 0')  # количество пропущенных значений

,empty


4. Какой столбец содержит целевую переменную? Запишите его название.

Целевую переменную содержит столбец 'radiant_win'.

5. Забудем, что в выборке есть категориальные признаки, и попробуем обучить градиентный бустинг над деревьями на имеющейся матрице "объекты-признаки". Зафиксируйте генератор разбиений для кросс-валидации по 5 блокам (KFold), не забудьте перемешать при этом выборку (shuffle=True), поскольку данные в таблице отсортированы по времени, и без перемешивания можно столкнуться с нежелательными эффектами при оценивании качества. Оцените качество градиентного бустинга (GradientBoostingClassifier) с помощью данной кросс-валидации, попробуйте при этом разное количество деревьев (как минимум протестируйте следующие значения для количества деревьев: 10, 20, 30). Долго ли настраивались классификаторы? Достигнут ли оптимум на испытанных значениях параметра n_estimators, или же качество, скорее всего, продолжит расти при дальнейшем его увеличении?

In [13]:
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import GridSearchCV 
from sklearn.model_selection  import KFold
from sklearn.metrics import roc_auc_score, make_scorer

In [14]:
gbclf = GradientBoostingClassifier()

In [15]:
scorer = make_scorer(roc_auc_score)

In [16]:
search = GridSearchCV(gbclf, {'n_estimators': [10, 20, 30, 40]}, 
                     cv=KFold(n_splits=5, shuffle=True),
                     scoring=scorer, n_jobs=-1, verbose=True) 

In [17]:
search.fit(train_X, train_y)

Fitting 5 folds for each of 4 candidates, totalling 20 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  20 out of  20 | elapsed:  2.2min finished
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\ensemble\_gb.py:1454: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


GridSearchCV(cv=KFold(n_splits=5, random_state=None, shuffle=True),
             error_score=nan,
             estimator=GradientBoostingClassifier(ccp_alpha=0.0,
                                                  criterion='friedman_mse',
                                                  init=None, learning_rate=0.1,
                                                  loss='deviance', max_depth=3,
                                                  max_features=None,
                                                  max_leaf_nodes=None,
                                                  min_impurity_decrease=0.0,
                                                  min_impurity_split=None,
                                                  min_samples_leaf=1,
                                                  min_samples_split=2,
                                                  min_weight_fraction_leaf=0.0,
                                                  n_estimators=100,
                    

In [18]:
search.best_score_

0.6355163813777166

In [19]:
search.best_params_

{'n_estimators': 40}

In [20]:
search.best_estimator_

GradientBoostingClassifier(ccp_alpha=0.0, criterion='friedman_mse', init=None,
                           learning_rate=0.1, loss='deviance', max_depth=3,
                           max_features=None, max_leaf_nodes=None,
                           min_impurity_decrease=0.0, min_impurity_split=None,
                           min_samples_leaf=1, min_samples_split=2,
                           min_weight_fraction_leaf=0.0, n_estimators=40,
                           n_iter_no_change=None, presort='deprecated',
                           random_state=None, subsample=1.0, tol=0.0001,
                           validation_fraction=0.1, verbose=0,
                           warm_start=False)

In [21]:
search.cv_results_

{'mean_fit_time': array([16.91861596, 36.33587518, 57.23186531, 61.68695898]),
 'std_fit_time': array([0.2818159 , 1.50417132, 0.65903527, 6.44932015]),
 'mean_score_time': array([0.07948451, 0.08919511, 0.08912253, 0.06481657]),
 'std_score_time': array([0.01045916, 0.01519173, 0.00523626, 0.01007109]),
 'param_n_estimators': masked_array(data=[10, 20, 30, 40],
              mask=[False, False, False, False],
        fill_value='?',
             dtype=object),
 'params': [{'n_estimators': 10},
  {'n_estimators': 20},
  {'n_estimators': 30},
  {'n_estimators': 40}],
 'split0_test_score': array([0.60779216, 0.62359421, 0.62873229, 0.63370142]),
 'split1_test_score': array([0.61180364, 0.62701831, 0.63734801, 0.64029484]),
 'split2_test_score': array([0.60578455, 0.62364464, 0.63036753, 0.63693972]),
 'split3_test_score': array([0.60731167, 0.62888637, 0.63251494, 0.63386548]),
 'split4_test_score': array([0.60909088, 0.62589671, 0.63123553, 0.63278045]),
 'mean_test_score': array([0.608

- Классификаторы настраивались довольно долго - 2.2 минуты
- Оптимум качества не достигнут на значениях n_estimators = [10, 20, 30]. Качество, скорее всего, продолжит расти, с увеличением количества деревьев.

Проверим качество модели на тестовой выборке:

In [22]:
features_test = features_test.fillna(0)

In [23]:
features_test.head()

,match_id,start_time,lobby_type,r1_hero,r1_level,r1_xp,r1_gold,r1_lh,r1_kills,r1_deaths,...,radiant_ward_sentry_count,radiant_first_ward_time,dire_bottle_time,dire_courier_time,dire_flying_courier_time,dire_tpscroll_count,dire_boots_count,dire_ward_observer_count,dire_ward_sentry_count,dire_first_ward_time
0,6,1430287923,0,93,4,1103,1089,8,0,1,...,0,12.0,247.0,-86.0,272.0,3,4,2,0,118.0
1,7,1430293357,1,20,2,556,570,1,0,0,...,2,-29.0,168.0,-54.0,0.0,3,2,2,1,16.0
2,10,1430301774,1,112,2,751,808,1,0,0,...,1,-22.0,46.0,-87.0,186.0,1,3,3,0,-34.0
3,13,1430323933,1,27,3,708,903,1,1,1,...,2,-49.0,30.0,-89.0,210.0,3,4,2,1,-26.0
4,16,1430331112,1,39,4,1259,661,4,0,0,...,0,36.0,180.0,-86.0,180.0,1,3,2,1,-33.0


In [24]:
search.best_estimator_.predict_proba(features_test)[:, 1]

array([0.50072336, 0.5986797 , 0.47285865, ..., 0.30053628, 0.35252806,
       0.5104872 ])

Этот результат надо записать и отправить на kaggle, чтобы узнать насколько он корректен, но доступ к соревнованию ограничен :(

# Что указать в отчете

В отчете по данному этапу вы должны ответить на следующие вопросы:

1. Какие признаки имеют пропуски среди своих значений? Что могут означать пропуски в этих признаках (ответьте на этот вопрос для двух любых признаков)?
    - Признак first_blood_time содержит пустые значения, т.к событие "первая кровь" не произошло за первые 5 минут.
    - Признак first_blood_team также содержит пустые значения, т.к событие "первая кровь" не произошло за первые 5 минут.
2. Как называется столбец, содержащий целевую переменную?
    - Целевую переменную содержит столбец 'radiant_win'.
3. Как долго проводилась кросс-валидация для градиентного бустинга с 30 деревьями? Инструкцию по измерению времени можно найти ниже по тексту. Какое качество при этом получилось? Напомним, что в данном задании мы используем метрику качества AUC-ROC.
    - Кросс-валидация производилась довольно долго. При 30 деревьях, качество равняется 63 %.
4. Имеет ли смысл использовать больше 30 деревьев в градиентном бустинге? Что бы вы предложили делать, чтобы ускорить его обучение при увеличении количества деревьев?
    - Думаю, что нет, потому что выиграное качесто будет маленьким, а рассчеты будут довольно сложными. Либо можно использовать случайную выборку объектов для уменьшения времени расчетов. Но это может привести к определенному снижению качества. Также можно урезать глубину деревьев.

# Подход 2: логистическая регрессия

Линейные методы работают гораздо быстрее композиций деревьев, поэтому кажется разумным воспользоваться именно ими для ускорения анализа данных. Одним из наиболее распространенных методов для классификации является логистическая регрессия.

Важно: не забывайте, что линейные алгоритмы чувствительны к масштабу признаков! Может пригодиться sklearn.preprocessing.StandardScaler.

In [25]:
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler

1. Оцените качество логистической регрессии (sklearn.linear_model.LogisticRegression с L2-регуляризацией) с помощью кросс-валидации по той же схеме, которая использовалась для градиентного бустинга. Подберите при этом лучший параметр регуляризации (C). Какое наилучшее качество у вас получилось? Как оно соотносится с качеством градиентного бустинга? Чем вы можете объяснить эту разницу? Быстрее ли работает логистическая регрессия по сравнению с градиентным бустингом?

In [26]:
scaler = StandardScaler()
std_train_X = scaler.fit_transform(train_X)
std_test_X = scaler.transform(features_test)

In [27]:
lrclf = LogisticRegression(penalty='l2')

In [28]:
search_2 = GridSearchCV(lrclf, {'C': np.arange(0, 1.05, 0.025)}, 
                     cv=KFold(n_splits=5, shuffle=True),
                     scoring=scorer, n_jobs=-1, verbose=True) 

In [29]:
search_2.fit(std_train_X, train_y)

Fitting 5 folds for each of 42 candidates, totalling 210 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:   11.4s
[Parallel(n_jobs=-1)]: Done 184 tasks      | elapsed:  1.1min
[Parallel(n_jobs=-1)]: Done 210 out of 210 | elapsed:  1.2min finished
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\utils\validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


GridSearchCV(cv=KFold(n_splits=5, random_state=None, shuffle=True),
             error_score=nan,
             estimator=LogisticRegression(C=1.0, class_weight=None, dual=False,
                                          fit_intercept=True,
                                          intercept_scaling=1, l1_ratio=None,
                                          max_iter=100, multi_class='auto',
                                          n_jobs=None, penalty='l2',
                                          random_state=None, solver='lbfgs',
                                          tol=0.0001, verbose=0,
                                          warm_start=False),
             iid='deprecated', n_jobs...
             param_grid={'C': array([0.   , 0.025, 0.05 , 0.075, 0.1  , 0.125, 0.15 , 0.175, 0.2  ,
       0.225, 0.25 , 0.275, 0.3  , 0.325, 0.35 , 0.375, 0.4  , 0.425,
       0.45 , 0.475, 0.5  , 0.525, 0.55 , 0.575, 0.6  , 0.625, 0.65 ,
       0.675, 0.7  , 0.725, 0.75 , 0.775, 0.8  , 0.82

In [30]:
search_2.best_score_

0.6540043742376404

In [31]:
search_2.best_params_

{'C': 0.05}

In [32]:
search_2.best_estimator_

LogisticRegression(C=0.05, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

- Наилучшее качество получилось равным 65%.
- Это выше качества градиентного бустинга.
- Эта разница может объясняться тем, что градиентный бустинг хуже справляется с разреженными данными.
- Логистическая регрессия работает на порядок быстрее по сравнению с градиентным бустингом.

2. Среди признаков в выборке есть категориальные, которые мы использовали как числовые, что вряд ли является хорошей идеей. Категориальных признаков в этой задаче одиннадцать: lobby_type и r1_hero, r2_hero, ..., r5_hero, d1_hero, d2_hero, ..., d5_hero. Уберите их из выборки, и проведите кросс-валидацию для логистической регрессии на новой выборке с подбором лучшего параметра регуляризации. Изменилось ли качество? Чем вы можете это объяснить?

In [33]:
train_X_without_categogials = train_X
train_X_without_categogials = train_X_without_categogials.drop(['lobby_type', 'r1_hero', 'r2_hero', 'r3_hero',
                                                                'r4_hero', 'r5_hero', 'd1_hero', 'd2_hero', 
                                                                 'd3_hero', 'd4_hero', 'd5_hero'], axis=1)
train_X_without_categogials.head()

,match_id,start_time,r1_level,r1_xp,r1_gold,r1_lh,r1_kills,r1_deaths,r1_items,r2_level,...,radiant_ward_sentry_count,radiant_first_ward_time,dire_bottle_time,dire_courier_time,dire_flying_courier_time,dire_tpscroll_count,dire_boots_count,dire_ward_observer_count,dire_ward_sentry_count,dire_first_ward_time
0,0,1430198770,5,2098,1489,20,0,0,7,3,...,0,35.0,103.0,-84.0,221.0,3,4,2,2,-52.0
1,1,1430220345,4,1188,1033,9,0,1,12,4,...,0,-20.0,149.0,-84.0,195.0,5,4,3,1,-5.0
2,2,1430227081,4,1319,1270,22,0,0,12,3,...,1,-39.0,45.0,-77.0,221.0,3,4,3,1,13.0
3,3,1430263531,4,1779,1056,14,0,0,5,2,...,0,-30.0,124.0,-80.0,184.0,0,4,2,0,27.0
4,4,1430282290,4,1431,1090,8,1,0,8,2,...,0,46.0,182.0,-80.0,225.0,6,3,3,0,-16.0


In [34]:
train_X_without_categogials.shape

(97230, 92)

In [35]:
std_train_X = scaler.fit_transform(train_X_without_categogials)

In [36]:
search_3 = GridSearchCV(lrclf, {'C': np.arange(0, 1.05, 0.025)}, 
                     cv=KFold(n_splits=5, shuffle=True),
                     scoring=scorer, n_jobs=-1, verbose=True) 

In [37]:
search_3.fit(train_X_without_categogials, train_y)

Fitting 5 folds for each of 42 candidates, totalling 210 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:    3.6s
[Parallel(n_jobs=-1)]: Done 184 tasks      | elapsed:   19.7s
[Parallel(n_jobs=-1)]: Done 210 out of 210 | elapsed:   22.3s finished
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\utils\validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


GridSearchCV(cv=KFold(n_splits=5, random_state=None, shuffle=True),
             error_score=nan,
             estimator=LogisticRegression(C=1.0, class_weight=None, dual=False,
                                          fit_intercept=True,
                                          intercept_scaling=1, l1_ratio=None,
                                          max_iter=100, multi_class='auto',
                                          n_jobs=None, penalty='l2',
                                          random_state=None, solver='lbfgs',
                                          tol=0.0001, verbose=0,
                                          warm_start=False),
             iid='deprecated', n_jobs...
             param_grid={'C': array([0.   , 0.025, 0.05 , 0.075, 0.1  , 0.125, 0.15 , 0.175, 0.2  ,
       0.225, 0.25 , 0.275, 0.3  , 0.325, 0.35 , 0.375, 0.4  , 0.425,
       0.45 , 0.475, 0.5  , 0.525, 0.55 , 0.575, 0.6  , 0.625, 0.65 ,
       0.675, 0.7  , 0.725, 0.75 , 0.775, 0.8  , 0.82

In [38]:
search_3.best_score_

0.5

In [39]:
search_3.best_params_

{'C': 0.025}

In [40]:
search_3.best_estimator_

LogisticRegression(C=0.025, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

- Качество довольно сильно ухудшилось.
- Это можно объяснить тем, что данные признаки сильно влияли на исход матча.

3. На предыдущем шаге мы исключили из выборки признаки rM_hero и dM_hero, которые показывают, какие именно герои играли за каждую команду. Это важные признаки — герои имеют разные характеристики, и некоторые из них выигрывают чаще, чем другие. Выясните из данных, сколько различных идентификаторов героев существует в данной игре (вам может пригодиться фукнция unique или value_counts).

In [41]:
train_X[['r1_hero', 'r2_hero', 'r3_hero', 'r4_hero', 'r5_hero', 
         'd1_hero', 'd2_hero', 'd3_hero', 'd4_hero', 'd5_hero']].nunique()

r1_hero    108
r2_hero    108
r3_hero    108
r4_hero    108
r5_hero    108
d1_hero    108
d2_hero    108
d3_hero    108
d4_hero    108
d5_hero    108
dtype: int64

- В игре существует 108 различных идентификаторов героев.

In [42]:
train_X[['r1_hero', 'r2_hero', 'r3_hero', 'r4_hero', 'r5_hero', 
         'd1_hero', 'd2_hero', 'd3_hero', 'd4_hero', 'd5_hero']].max()  # Максимальный идентификатор героя

r1_hero    112
r2_hero    112
r3_hero    112
r4_hero    112
r5_hero    112
d1_hero    112
d2_hero    112
d3_hero    112
d4_hero    112
d5_hero    112
dtype: int64

4. Воспользуемся подходом "мешок слов" для кодирования информации о героях. Пусть всего в игре имеет N различных героев. Сформируем N признаков, при этом i-й будет равен нулю, если i-й герой не участвовал в матче; единице, если i-й герой играл за команду Radiant; минус единице, если i-й герой играл за команду Dire. Ниже вы можете найти код, который выполняет данной преобразование. Добавьте полученные признаки к числовым, которые вы использовали во втором пункте данного этапа.

In [43]:
N = 112
X_pick = np.zeros((train_X.shape[0], N))

for i, match_id in enumerate(train_X.index):
    for p in range(5):
        X_pick[i, train_X.loc[match_id, 'r%d_hero' % (p+1)]-1] = 1
        X_pick[i, train_X.loc[match_id, 'd%d_hero' % (p+1)]-1] = -1

In [44]:
X_pick.shape

(97230, 112)

In [45]:
train_X_without_categogials = train_X_without_categogials.fillna(0)

In [46]:
train_X_without_categogials.values.shape

(97230, 92)

In [47]:
train_X_with_categorials = np.hstack((train_X_without_categogials.values, X_pick))

In [48]:
train_X_with_categorials.shape

(97230, 204)

5. Проведите кросс-валидацию для логистической регрессии на новой выборке с подбором лучшего параметра регуляризации. Какое получилось качество? Улучшилось ли оно? Чем вы можете это объяснить?

In [49]:
std_train_X_with_categorials = scaler.fit_transform(train_X_with_categorials)

In [50]:
search_4 = GridSearchCV(lrclf, {'C': np.arange(0, 1.05, 0.025)}, 
                     cv=KFold(n_splits=5, shuffle=True),
                     scoring=scorer, n_jobs=-1, verbose=True) 

In [51]:
search_4.fit(std_train_X_with_categorials, train_y)

Fitting 5 folds for each of 42 candidates, totalling 210 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:   24.2s
[Parallel(n_jobs=-1)]: Done 184 tasks      | elapsed:  2.3min
[Parallel(n_jobs=-1)]: Done 210 out of 210 | elapsed:  2.6min finished
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\utils\validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


GridSearchCV(cv=KFold(n_splits=5, random_state=None, shuffle=True),
             error_score=nan,
             estimator=LogisticRegression(C=1.0, class_weight=None, dual=False,
                                          fit_intercept=True,
                                          intercept_scaling=1, l1_ratio=None,
                                          max_iter=100, multi_class='auto',
                                          n_jobs=None, penalty='l2',
                                          random_state=None, solver='lbfgs',
                                          tol=0.0001, verbose=0,
                                          warm_start=False),
             iid='deprecated', n_jobs...
             param_grid={'C': array([0.   , 0.025, 0.05 , 0.075, 0.1  , 0.125, 0.15 , 0.175, 0.2  ,
       0.225, 0.25 , 0.275, 0.3  , 0.325, 0.35 , 0.375, 0.4  , 0.425,
       0.45 , 0.475, 0.5  , 0.525, 0.55 , 0.575, 0.6  , 0.625, 0.65 ,
       0.675, 0.7  , 0.725, 0.75 , 0.775, 0.8  , 0.82

In [52]:
search_4.best_score_

0.6818127645253138

In [53]:
search_4.best_params_

{'C': 0.025}

In [54]:
search_4.best_estimator_

LogisticRegression(C=0.025, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

- Получили качество 68 %.
- Качество улучшилось и пока является наилучим.
- Теперь логистическая регрессия корректно работает с важными категориальными признаками. Отсюда и прирост в качестве.

6. Постройте предсказания вероятностей победы команды Radiant для тестовой выборки с помощью лучшей из изученных моделей (лучшей с точки зрения AUC-ROC на кросс-валидации). Убедитесь, что предсказанные вероятности адекватные — находятся на отрезке [0, 1], не совпадают между собой (т.е. что модель не получилась константной).

In [55]:
features_test_without_categorials = features_test.drop(['lobby_type', 'r1_hero', 'r2_hero', 'r3_hero',
                                                                'r4_hero', 'r5_hero', 'd1_hero', 'd2_hero', 
                                                                 'd3_hero', 'd4_hero', 'd5_hero'], axis=1)

In [56]:
features_test_without_categorials.shape

(17177, 92)

In [57]:
test_X_pick = np.zeros((features_test.shape[0], N))

for i, match_id in enumerate(features_test.index):
    for p in range(5):
        X_pick[i, features_test.loc[match_id, 'r%d_hero' % (p+1)]-1] = 1
        X_pick[i, features_test.loc[match_id, 'd%d_hero' % (p+1)]-1] = -1

In [58]:
test_X_pick.shape

(17177, 112)

In [59]:
features_test_without_categorials = features_test_without_categorials.fillna(0)

In [60]:
test_X_with_categorials = np.hstack((features_test_without_categorials.values, test_X_pick))

In [61]:
test_X_with_categorials.shape

(17177, 204)

In [62]:
std_features_test = scaler.transform(test_X_with_categorials)

In [63]:
search_4.best_estimator_.predict_proba(std_features_test)[:, 1]

array([0.52741204, 0.66585113, 0.32638911, ..., 0.23635391, 0.36967625,
       0.53823931])

Этот результат тоже надо записать и отправить на kaggle, чтобы узнать насколько он корректен, но доступ к соревнованию ограничен :(

In [64]:
search_4.best_estimator_.predict_proba(std_features_test)[:, 1].min()

0.005626408444192738

In [65]:
search_4.best_estimator_.predict_proba(std_features_test)[:, 1].max()

0.99422065533639

## Что указать в отчете

В отчете по данному этапу вы должны ответить на следующие вопросы:

1. Какое качество получилось у логистической регрессии над всеми исходными признаками? Как оно соотносится с качеством градиентного бустинга? Чем вы можете объяснить эту разницу? Быстрее ли работает логистическая регрессия по сравнению с градиентным бустингом?
    - Наилучшее качество получилось равным 65%.
    - Это выше качества градиентного бустинга.
    - Эта разница может объясняться тем, что градиентный бустинг хуже справляется с разреженными данными.
    - Логистическая регрессия работает на порядок быстрее по сравнению с градиентным бустингом.
2. Как влияет на качество логистической регрессии удаление категориальных признаков (укажите новое значение метрики качества)? Чем вы можете объяснить это изменение?
    - Качество довольно сильно ухудшилось. Новое значение метрики 50 %.
    - Это можно объяснить тем, что данные признаки сильно влияли на исход матча.
3. Сколько различных идентификаторов героев существует в данной игре?
    - В игре существует 108 различных идентификаторов героев.
4. Какое получилось качество при добавлении "мешка слов" по героям? Улучшилось ли оно по сравнению с предыдущим вариантом? Чем вы можете это объяснить?
    - Получили качество 68 %.
    - Качество улучшилось и пока является наилучим.
    - Теперь логистическая регрессия корректно работает с важными категориальными признаками. Отсюда и прирост в качестве.
5. Какое минимальное и максимальное значение прогноза на тестовой выборке получилось у лучшего из алгоритмов?
    - 0.005626408444192738 - минимальное. 
    - 0.99422065533639 - максимальное.